In [ ]:
# SPLITING DATA FROM DELIMITED TEXT FILE
# AND SORTING IT IN DATABASE

In [13]:
#CONNECTION TO THE DATABASE
import mysql.connector as mysql
from mysql.connector import Error

try:
    instance = mysql.connect(host='localhost',database='incubyte',user='root',password='mysql@123')
    if instance.is_connected():
        db_info=instance.get_server_info()
        print("Connected...",db_info)
except Error:
    print("Error occured...")

Connected... 8.0.23


In [15]:
mycursor=instance.cursor(buffered=True)

#FUNCTION FOR CREATING DYNAMIC COUNTRY TABLE
def creation(country):
    mycursor.execute("""CREATE TABLE """ +country+ """ (`CustomerName` VARCHAR(255) NOT NULL,
                          `CustomerID` VARCHAR(18) NOT NULL,
                          `CustomerOpenDate` DATE NOT NULL,
                          `LastConsultedDate` DATE NULL,
                          `VaccinationType` CHAR(5) NULL,
                          `DoctorConsulted` CHAR(255) NULL,
                          `State` CHAR(5) NULL,
                          `Country` CHAR(5) NULL,
                          `PostCode` INT(5) NULL,
                          `DOB` DATE NULL,
                          `ActiveCustomer` CHAR(1) NULL,
                          PRIMARY KEY (`CustomerName`))""")
    print("New Country table created successfully...")

#FUNCTION FOR INSERTING DATA IN COUNTRY TABLE
def insertion(country,data):
    InsertQuery = """INSERT INTO """+ country+"""(CustomerName,CustomerID ,CustomerOpenDate ,LastConsultedDate,VaccinationType ,DoctorConsulted ,State ,Country,PostCode,DOB,ActiveCustomer) 
                    VALUES (%s, %s, %s, %s, %s,%s,%s,%s,%s,%s,%s)"""
    mycursor.execute(InsertQuery,data)
    print("Data to new country table is inserted successfully...")

# SPLITING DATA FROM DELIMITED FILE
try: 
    with open('DelimitedData.txt') as f:
        lines = f.readlines()
        for line in lines:       
            if (line.split('|')[1]=='D'):
                name=line.split('|')[2]
                cid=line.split('|')[3]
                opendate=line.split('|')[4]
                closedate=line.split('|')[5]
                vacctype=line.split('|')[6]
                dr=line.split('|')[7]
                state=line.split('|')[8]
                country=line.split('|')[9]
                pc=0
                dob=line.split('|')[10][4:8]+"/"+line.split('|')[10][2:4]+"/"+line.split('|')[10][0:2]
                activeCust=line.split('|')[11]

                print("CustomerName: ",name,"\n","CustomerID: ",cid,"\n","OpenDate: ",opendate[0:4],"/",opendate[4:6],"/",opendate[6:8],"\n","CloseDate: ",closedate[0:4],"/",closedate[4:6],"/",closedate[6:8],"\n","VaccinationType: ",vacctype,"\n","DoctorConsulted: ",dr,"\n","State: ",state,"\n","Country: ",country,"\n","DOB: ",dob,"\n","PostCode: ",pc,"\n","ActiveCustomer: ",activeCust)
                
                try:
                    data=(name, cid,opendate,closedate,vacctype,dr,state,country,pc,dob,activeCust)
                    #INSERT QUERY IN CUSTOMER TABLE
                    insque = """INSERT INTO customer(CustomerName,CustomerID ,CustomerOpenDate ,LastConsultedDate,VaccinationType ,DoctorConsulted ,State ,Country,PostCode,DOB,ActiveCustomer) 
                    VALUES (%s, %s, %s, %s, %s,%s,%s,%s,%s,%s,%s)"""
                    mycursor.execute(insque,data)
                    print("Data inserted in Customer table successfully...")

                    #CHECKING IF COUNTRY TABLE EXISITS OR NOT!  
                    checkpoint=0
                    showQuery = "show tables;"
                    mycursor.execute(showQuery)
                    output =mycursor.fetchall()
                    for i in output:
                        if (i[0].upper()==country):
                            checkpoint=1
                            break

                    if (checkpoint==0):
                        #IF TABLE DOES NOT EXISITS THEN IT WILL CREATE A NEW TABLE OF COUNTRY NAME AND INSERT DATA IN IT!
                        creation(country)   
                        insertion(country,data)
                                                
                    else:
                        #IF TABLE EXISITS THEN IT WILL INSERT DATA IN IT!
                        insertion(country,data)
                
                except mysql.IntegrityError:
                    print(name," already exists in the Customer table")
                except mysql.DataError:
                    print("Data is not formatted as given in the database")
                except mysql.ProgrammingError:
                    print("Not enough parameters for the SQL statement")    

except IndexError:
    print("Your file is not formatted properly")
except FileNotFoundError:
    print("Your file is not available on the location")

finally:
    instance.commit()  
    mycursor.close()
    print("MySQL connection is closed...")

CustomerName:  Alex 
 CustomerID:  123457 
 OpenDate:  2010 / 10 / 12 
 CloseDate:  2012 / 10 / 13 
 VaccinationType:  MVD 
 DoctorConsulted:  Paul 
 State:  SA 
 Country:  USA 
 DOB:  1987/03/06 
 PostCode:  0 
 ActiveCustomer:  A

Data inserted in Customer table successfully...
New Country table created successfully...
Data to new country table is inserted successfully...
CustomerName:  John 
 CustomerID:  123458 
 OpenDate:  2010 / 10 / 12 
 CloseDate:  2012 / 10 / 13 
 VaccinationType:  MVD 
 DoctorConsulted:  Paul 
 State:  TN 
 Country:  IND 
 DOB:  1987/03/06 
 PostCode:  0 
 ActiveCustomer:  A

Data inserted in Customer table successfully...
New Country table created successfully...
Data to new country table is inserted successfully...
CustomerName:  Mathew 
 CustomerID:  123459 
 OpenDate:  2010 / 10 / 12 
 CloseDate:  2012 / 10 / 13 
 VaccinationType:  MVD 
 DoctorConsulted:  Paul 
 State:  WAS 
 Country:  PHIL 
 DOB:  1987/03/06 
 PostCode:  0 
 ActiveCustomer:  A

Data inse